In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader,TensorDataset

# reading the data
 

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!unzip '/content/gdrive/MyDrive/MSC - intelligent systems/Final Project/Data/FINAL AFTER PREPROCESS/final_codon_dataset.csv.zip'

Archive:  /content/gdrive/MyDrive/MSC - intelligent systems/Final Project/Data/FINAL AFTER PREPROCESS/final_codon_dataset.csv.zip
  inflating: final_codon_dataset.csv  


In [72]:
data = pd.read_csv('final_codon_dataset.csv')

In [11]:
data.shape

(472336, 100)

In [73]:
data.pop('Unnamed: 0')
data.pop('Unnamed: 0.1')
data = data[list(data.columns)[8:]]
data.info()
data.shape


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472336 entries, 0 to 472335
Data columns (total 91 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   GC%       472336 non-null  float64
 1   GC1%      472336 non-null  float64
 2   GC2%      472336 non-null  float64
 3   GC3%      472336 non-null  float64
 4   %A        472336 non-null  float64
 5   %G        472336 non-null  float64
 6   %C        472336 non-null  float64
 7   %T        472336 non-null  float64
 8   %TT       472336 non-null  float64
 9   %TG       472336 non-null  float64
 10  %TC       472336 non-null  float64
 11  %TA       472336 non-null  float64
 12  %GT       472336 non-null  float64
 13  %GG       472336 non-null  float64
 14  %GC       472336 non-null  float64
 15  %GA       472336 non-null  float64
 16  %CT       472336 non-null  float64
 17  %CG       472336 non-null  float64
 18  %CC       472336 non-null  float64
 19  %CA       472336 non-null  float64
 20  %AT 

(472336, 91)

# VAE encoder


In [74]:
class Encoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.fc1 = nn.Linear(91, 46)
        self.fc2 = nn.Linear(46, 23)
        self.fc3_mean = nn.Linear(23, latent_dim)
        self.fc3_logvar = nn.Linear(23, latent_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = x.view(x.size(0), -1)
        mean = self.fc3_mean(x)
        logvar = self.fc3_logvar(x)
        return mean, logvar


class Decoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.fc1 = nn.Linear(latent_dim, 23)
        self.fc2 = nn.Linear(23, 46)
        self.fc3 = nn.Linear(46, 91)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x.view(x.size(0), -1)


class VAE(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.encoder = Encoder(latent_dim)
        self.decoder = Decoder(latent_dim)

    def forward(self, x):
        mean, logvar = self.encoder(x)
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = eps * std + mean
        recon_x = self.decoder(z)
        return recon_x, mean, logvar


def loss_function(recon_x, x, mean, logvar):
    mse_loss = nn.MSELoss(reduction='sum')
    mse = mse_loss(recon_x, x)
    kl_loss = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp())
    return mse + kl_loss


# Data loading and optimizing

In [84]:
device = torch.device('cuda')

tensor = torch.tensor(data.values).float()

# Create a TensorDataset from the tensor
dataset = TensorDataset(tensor)

# Create a DataLoader from the dataset
batch_size = 64
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last = True)

In [80]:
data.shape

(472336, 91)

In [78]:
latent_dim = 10
vae = VAE(latent_dim).to(device)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)

In [ ]:
num_epochs = 50
for epoch in range(num_epochs):
  total_loss = 0

  for batch_idx, data1 in enumerate(loader):
    data1 = [feature.to(device) for feature in data1]
    optimizer.zero_grad()
    recon_batch, mu, logvar = vae(torch.cat(data1, dim=1))
    loss = loss_function(recon_batch, torch.cat(data1, dim=1), mu, logvar)
    loss.backward()
    total_loss += loss.item()
    optimizer.step()
    avg_loss = total_loss / len(loader.dataset)
  print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, avg_loss))